# Model Inference

## 1. Installs and Imports

In [1]:
!pip install datasets transformers seqeval

     |████████████████████████████████| 306 kB 5.5 MB/s 
     |████████████████████████████████| 3.4 MB 29.3 MB/s 
     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 67 kB 4.0 MB/s 
     |████████████████████████████████| 243 kB 71.5 MB/s 
     |████████████████████████████████| 1.1 MB 70.4 MB/s 
     |████████████████████████████████| 133 kB 58.0 MB/s 
     |████████████████████████████████| 895 kB 34.6 MB/s 
     |████████████████████████████████| 3.3 MB 48.8 MB/s 
     |████████████████████████████████| 596 kB 64.8 MB/s 
     |████████████████████████████████| 271 kB 53.3 MB/s 
     |████████████████████████████████| 192 kB 76.7 MB/s 
     |████████████████████████████████| 160 kB 69.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=4956a1b2ac6504b0465841c86d48f0f60c3b65d9161527add88988fdc6e91272
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72

In [2]:
import os
import random
import transformers
import pandas as pd
import numpy as np
from datetime import date
from pathlib import Path
from google.colab import drive
from seqeval.metrics import accuracy_score
from IPython.display import display, HTML
from collections import defaultdict, Counter, OrderedDict
from datasets import load_dataset, load_metric, load_from_disk, ClassLabel, Sequence
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification

#scraping
import requests
from bs4 import BeautifulSoup

import spacy
from spacy import displacy

#inference
import torch
from transformers import pipeline

print(transformers.__version__)

4.15.0


In [3]:
system = "COLAB" #["AWS", "COLAB"]

In [5]:
if system=="COLAB":
  drive.mount("/content/gdrive")
  DATA_DIR = os.path.join("/content/gdrive/Shared drives/", "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data")
  MODEL_DIR = os.path.join("/content/gdrive/Shared drives/", "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Models")
  RESULTS_DIR = os.path.join("/content/gdrive/Shared drives/", "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Models/Metrics")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
print("Data Folder: {}".format(DATA_DIR))
print(os.listdir(DATA_DIR)[:3])
print("Model Folder: {}".format(MODEL_DIR))
print(os.listdir(MODEL_DIR)[:3])

Data Folder: /content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data
['line_by_line_NER_data_sampled_09062020_more_ents.csv', 'line_by_line_NER_data_sampled_12062020_more_ents.csv', 'line_by_line_NER_data_sampled.csv']
Model Folder: /content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Models
['distilbert-base-uncased-finetuned-ner-govuk-14-01-2022-validated_train', 'distilbert-base-uncased-finetuned-ner-govuk-14-01-2022-unvalidated_train_sample']


## 2. Scrape Govuk Content

In [7]:
def get_page_soup(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  return soup

In [8]:
def get_sents_from_soup(soup):
  body = soup.findAll(attrs={"class":"gem-c-govspeak"})
  sents = [i.text.split('\n') for i in body]
  sents_clean = [list(filter(None, i)) for i in sents]
  return sents_clean

In [9]:
def url_get_sents(url):
  soup = get_page_soup(url)
  sents_clean = get_sents_from_soup(soup)
  return sents_clean

In [10]:
result = url_get_sents('https://www.gov.uk/student-visa')

In [11]:
result

[['You can apply for a Student visa to study in the UK if you’re 16 or over and you:',
  'have been offered a place on a course by a licensed student sponsor',
  'have enough money to support yourself and pay for your course - the amount will vary depending on your circumstances',
  'can speak, read, write and understand English',
  'have consent from your parents if you’re 16 or 17 - you’ll need evidence of this when you apply',
  'If you’re 16 or 17 and you want to study at an independent school in the UK, you may be eligible for a Child Student visa instead.',
  'This visa has replaced the Tier 4 (General) student visa.',
  'If you or your family are from the EU, Switzerland, Norway, Iceland or Liechtenstein',
  'If you or your family member started living in the UK by 31 December 2020, you may be able to apply to the free EU Settlement Scheme.',
  'The deadline to apply was 30 June 2021 for most people. You can still apply if either:',
  'you have a later deadline - for example, yo

## 3. Do Inference

### 3A. Load Model

Load model from local


In [12]:
# MODEL_DIR 
# model_name = "distilbert-base-uncased"
# task = "ner"
# dataset_name = "govuk"
# req_date = "13-12-2021"
# dataset_type = 'FULL'
# chkpoint = 'checkpoint-73500'

In [13]:
# OUTPUT_PATH = f"{MODEL_DIR}/{model_name}-finetuned-{task}-{dataset_name}-{dataset_type}-{req_date}/{chkpoint}"
# OUTPUT_PATH

In [44]:
OUTPUT_PATH = '/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Models/distilbert-base-uncased-finetuned-ner-govuk-14-01-2022-validated_train/checkpoint-1000'

In [45]:
os.listdir(OUTPUT_PATH)

['config.json',
 'pytorch_model.bin',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'vocab.txt',
 'tokenizer.json',
 'training_args.bin',
 'optimizer.pt',
 'scheduler.pt',
 'trainer_state.json',
 'rng_state.pth']

Load model and tokeniser

In [46]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

In [47]:
model = AutoModelForTokenClassification.from_pretrained(OUTPUT_PATH)

In [48]:
tokenizer = AutoTokenizer.from_pretrained(OUTPUT_PATH)

### 3B. Hugging Face Pipelines

Use Huggingface Pipelines

In [49]:
sequence = "You must be at least 17 years old to have a drivers licence " \
"failure to provide this certificate will mean imprisonment in the UK and barring from countries like EU and US"

In [50]:
# set up pipeline with model and tokeniser
token_classifier = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [51]:
print(sequence)
print(len(sequence))

You must be at least 17 years old to have a drivers licence failure to provide this certificate will mean imprisonment in the UK and barring from countries like EU and US
170


In [52]:
result = token_classifier(sequence)

In [53]:
result

[{'end': 59,
  'entity_group': 'FORM',
  'score': 0.61682093,
  'start': 44,
  'word': 'drivers licence'},
 {'end': 95,
  'entity_group': 'FORM',
  'score': 0.57401067,
  'start': 84,
  'word': 'certificate'},
 {'end': 128,
  'entity_group': 'LOC',
  'score': 0.9417072,
  'start': 126,
  'word': 'uk'},
 {'end': 155,
  'entity_group': 'LOC',
  'score': 0.64312714,
  'start': 146,
  'word': 'countries'},
 {'end': 163,
  'entity_group': 'LOC',
  'score': 0.65338796,
  'start': 161,
  'word': 'eu'},
 {'end': 170,
  'entity_group': 'LOC',
  'score': 0.42156848,
  'start': 168,
  'word': 'us'}]

Now try with gov.uk outputs

In [54]:
page_sents = url_get_sents(url='https://www.gov.uk/marriage-visa/documents-you-will-need')

In [55]:
page_sents

[['You must provide a passport or travel document. Your passport should be valid for the whole of your stay in the UK and contain a blank page for your visa.',
  'You can supply the following to support your application:',
  'details of the marriage or civil partnership and proof that you’ve paid money for some of its costs',
  'proof that you’re planning to get married in the UK, for example a booking confirmation or emails between you and the venue',
  'See the full list of documents you can provide to prove your eligibility.',
  'You’ll need to provide a certified translation of any documents that are not in English or Welsh.',
  'If you’ve been married before',
  'You’ll need to show proof that you’re free to marry or enter into a civil partnership again, for example a:',
  'decree absolute',
  'death certificate of a previous partner',
  'You may need to provide additional documents depending on your circumstances.']]

In [56]:
sent_len = 0
ners = []
for i in page_sents[0]:
  result = token_classifier(i)
  for j in result:
    j['start'] += sent_len
    j['end'] += sent_len
  sent_len += len(i) + 1
  ners.append(result)

In [57]:
ners_flat = [item for sublist in ners for item in sublist]

In [58]:
ners_flat

[{'end': 27,
  'entity_group': 'FORM',
  'score': 0.5673246,
  'start': 19,
  'word': 'passport'},
 {'end': 46,
  'entity_group': 'FORM',
  'score': 0.66085386,
  'start': 31,
  'word': 'travel document'},
 {'end': 114,
  'entity_group': 'LOC',
  'score': 0.94163096,
  'start': 112,
  'word': 'uk'},
 {'end': 153,
  'entity_group': 'FORM',
  'score': 0.54074776,
  'start': 149,
  'word': 'visa'},
 {'end': 236,
  'entity_group': 'STATE',
  'score': 0.37200567,
  'start': 228,
  'word': 'marriage'},
 {'end': 257,
  'entity_group': 'STATE',
  'score': 0.44691032,
  'start': 240,
  'word': 'civil partnership'},
 {'end': 290,
  'entity_group': 'FINANCE',
  'score': 0.4894241,
  'start': 285,
  'word': 'money'},
 {'end': 364,
  'entity_group': 'LOC',
  'score': 0.9390909,
  'start': 362,
  'word': 'uk'},
 {'end': 400,
  'entity_group': 'FORM',
  'score': 0.58465934,
  'start': 380,
  'word': 'booking confirmation'},
 {'end': 410,
  'entity_group': 'CONTACT',
  'score': 0.6336969,
  'start': 4

In [59]:
res_ents = [(i['entity_group'], i['start'], i['end']) for i in ners_flat]
res_ents

[('FORM', 19, 27),
 ('FORM', 31, 46),
 ('LOC', 112, 114),
 ('FORM', 149, 153),
 ('STATE', 228, 236),
 ('STATE', 240, 257),
 ('FINANCE', 285, 290),
 ('LOC', 362, 364),
 ('FORM', 380, 400),
 ('CONTACT', 404, 410),
 ('LOC', 431, 436),
 ('CONTACT', 458, 467),
 ('STATE', 624, 631),
 ('STATE', 707, 724),
 ('FORM', 763, 780),
 ('PER', 795, 802),
 ('CONTACT', 838, 847)]

Stitch sents into one 'doc'

In [60]:
seq = ' '.join(page_sents[0])

In [61]:
seq

'You must provide a passport or travel document. Your passport should be valid for the whole of your stay in the UK and contain a blank page for your visa. You can supply the following to support your application: details of the marriage or civil partnership and proof that you’ve paid money for some of its costs proof that you’re planning to get married in the UK, for example a booking confirmation or emails between you and the venue See the full list of documents you can provide to prove your eligibility. You’ll need to provide a certified translation of any documents that are not in English or Welsh. If you’ve been married before You’ll need to show proof that you’re free to marry or enter into a civil partnership again, for example a: decree absolute death certificate of a previous partner You may need to provide additional documents depending on your circumstances.'

In [62]:
colors = {"ORG": "#7c5cdd", 
          "FORM": "#26e21c",
          "LOC": "#eee65c",
          "MONEY": "#80bab2",
          "SCHEME": "#b76d14",
          "DATE": "#bc8251",
          "STATE": "#bd4c33",
          "PER": "#c0970b",
          "FINANCE": "#debdd8",
          "FORM": "#48aba2",
          "EVENT": "#0a8dd9",
          "CONTACT": "#807388"}

In [63]:
def display_entities(text, entities):
  nlp = spacy.blank("en")
  doc = nlp(text)
  ents = []
  for ee in entities:
      ents.append(doc.char_span(ee[1], ee[2], ee[0]))
  doc.ents = ents
  options={'distance': 90, 'colors':colors}
  displacy.render(doc, style='ent', jupyter=True, options=options)

In [64]:
display_entities(text=seq, entities=res_ents)

In [65]:
def get_ners_and_flatten(sents):
  sent_len = 0
  ners = []
  for i in sents[0]:
    result = token_classifier(i)
    for j in result:
      j['start'] += sent_len
      j['end'] += sent_len
    sent_len += len(i) + 1
    ners.append(result)
  ners_flat = [item for sublist in ners for item in sublist]
  res_ents = [(i['entity_group'], i['start'], i['end']) for i in ners_flat]
  return res_ents

In [66]:
def stitch_sents(sent_list):
  seq = ' '.join(sent_list)
  return seq

In [67]:
def display_entities(text, entities):
  nlp = spacy.blank("en")
  doc = nlp(text)
  ents = []
  for ee in entities:
      ents.append(doc.char_span(ee[1], ee[2], ee[0]))
  doc.ents = ents
  options={'distance': 90, 'colors':colors}
  return displacy.render(doc, style='ent', jupyter=True, options=options)

## Pipeline

In [68]:
def url_to_spacy_viz(url):
  print(url)
  sents = url_get_sents(url)
  flat_ners = get_ners_and_flatten(sents)
  stitched = stitch_sents(sents[0])
  disp_ents = display_entities(stitched, flat_ners)
  return disp_ents

In [73]:
url_to_spacy_viz(url='https://www.gov.uk/hmrc-internal-manuals/tobacco-products-duty/tpd3180')

https://www.gov.uk/hmrc-internal-manuals/tobacco-products-duty/tpd3180


In [70]:
paths = ['/student-visa', 
         '/marriage-visa', 
         '/marriage-visa/eligibility', 
         '/marriage-visa/documents-you-will-need', 
         '/marriage-visa/apply', 
         '/guidance/covid-19-coronavirus-restrictions-what-you-can-and-cannot-do#what-has-changed',
         ]

In [71]:
for p in paths:
  url_p = f"http://www.gov.uk{p}"
  p_dash = p.replace("/", "_")
  fname = f"{DATA_DIR}/Images/img_{p_dash}"
  print(fname)
  disp_ents = url_to_spacy_viz(url_p)
  # output_path = Path(f"{DATA_DIR}/Images/img_{p_dash}.svg")
  # output_path.open("w", encoding="utf-8").write(disp_ents)

/content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/Images/img__student-visa
http://www.gov.uk/student-visa


TypeError: ignored